# Introduction
What's the last school shooting you remember? If you're on top of your news, then you'll know it was the Oshkosh, Washington shooting, about a week ago (from the time this was written). 
Do you remember the details, though?
Probably not.

What about the shooting the day right before Oshkosh? Or the four in November?
The problem is clear: school shootings are a regular occurrence and it's hard to keep track of them all, let alone the details of each one.
In fact, according to Wikipedia, there were 64 US school shootings from 2000-2009, 87 from 2010-2014, and 113 from 2015-November 2019. 
If these numbers seem lower than you expected, remember that these are ONLY school shootings, not mass shootings. The numbers there are staggering: 2,138 since 2013, roughly one a day. The number changes a bit depending on how you define a mass shooting, but either way the point remains: the number of shootings that occur in the U.S. is unacceptably high.

## Why are we doing analysis on this topic?
Our group recognizes that school shootings are an extremely sensitive topic, and we don't want anyone or any of their friends to ever go through an event like this. We want to research this topic and first find out if any specific shootings are getting more coverage than others because they should all have the same amount coverage from an ethical standpoint. 

For this project we want to focus on school shootings, as the definition of school shooting is much more clearly defined and can be analyzed easier with a smaller timeframe. Our goal is to find causes of the spike in shootings, especially in the past decade. From this coverage, we also want to determine if there are factors of the shooting that increased the chance of the shooting to occur. This does not mean that we are simply writing off other mass shootings that do not occur in schools, but we are trying to cover a topic that is still relevant to all of our lives.

It's important to note that none of us want to push any sort of policy or idea or agenda here: we want to stay as neutral as possible. The dataset that we are working with is relatively small so we will be careful in making certain assumptions.


# References
https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States
https://en.wikipedia.org/wiki/Mass_shootings_in_the_United_States
https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States_(before_2000)#20th_century
https://en.wikipedia.org/wiki/List_of_unsuccessful_attacks_related_to_schools



## Beginning our Analysis
Our goal is to figure out significant school shootings by getting compiled data from Wikipedia and recording certain characteristics about that specific event. With those characteristics, we want to scrape various data sources, such as Fox News, CNN, etc, and see if characteristics of the shooting affect the amount that it is reported among various source.
We recognize that because we have a very small dataset that we may not be able to come up with any significant conclusions, but we believe that it is a start to making steps towards doing so in identifying these reporting patterns.

In [1]:
import pandas as pd
import requests as rq
import numpy as np
import re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import plotly as py
import plotly.express as px
import time

### Determining how to begin parsing data
We want to figure out a way to identify the shootings and store those attributes of the shooting, such as city information, in order for us to use later. We could use some kind of database, but a pandas dataframe will suffice.

We first manually put in the wikipedia link that has all of the successful shootings from 2000 and on. We can discover the tables in the wikipedia article that lists all of the shootings and iterate through the columns and rows to add them to our dataframe. There was also a bit of data cleaning within the tables and the content, which we were able to clean up using text manipulation, such as regex.

In [2]:
def makeDF(header, body):
    sol = pd.DataFrame.from_records(body)
    sol.columns = header
    return sol.dropna()

def parseWiki(url):
    school_shooting_url = rq.get(url).text
    soup = BeautifulSoup(school_shooting_url)
    soup.prettify()
    # sortable wikitable is what wikipedia has their tables called
    tables = soup.find_all("table",{"class":"sortable wikitable"})
    header = []
    body = []
    # wikipedia separates by year chunks so we iterate through
    for table in tables:
        for row in table.find_all("tr"):
            temp = []
            if not header:
                for h in row.find_all("th"):
                    header.append(h.get_text().rstrip())
            for col in row.find_all("td"):
                cur = col.get_text()
                # want to get rid of wiki references
                cur = re.sub(r'\[.*\]', '', cur)
                cur = cur.replace("\n", " ")
                noline = cur.rstrip()
                temp.append(noline)
            body.append(temp)
    # the first is the header, so we take it out
    return header, body[1:]

header, body = parseWiki("https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States")
school_shootings = makeDF(header, body)

Waukesha and Oshkosh are very recent shootings: they occurred in the past week or so. Because of this, Wikipedia hasn't updated their city details and we are choosing to remove them from this analysis, as they are missing vital info. 
Bailey, Colorado and Great Mills, Maryland are very small communities, lacking Population statistics. Thus, we are also choosing to remove them from the dataset.

In [3]:
data = school_shootings.set_index("Location")
data = data.drop("Waukesha, Washington", axis=0)
data = data.drop("Oshkosh, Washington", axis=0)
data = data.drop("Bailey, Colorado", axis=0)
data = data.drop("Great Mills, Maryland", axis=0)
df = data.reset_index()

Here we have the earliest school shootings of our dataset, which date to earliest 2000.

In [4]:
df.head()

,Location,Date,Deaths,Injuries,Description
0,"Flint, Michigan","February 29, 2000",1,0,Shooting of Kayla Rolland: At Buell Elementary...
1,"Lake Worth, Florida","May 26, 2000",1,0,"13-year-old honor student, Nathaniel Brazill, ..."
2,"Seattle, Washington","June 28, 2000",2,0,58-year-old Director of the Division of Pathol...
3,"Fayetteville, Arkansas","August 28, 2000",2,0,"36-year-old James Easton Kelly, a PhD candidat..."
4,"New Orleans, Louisiana","September 26, 2000",0,2,"13 year-olds Darrel Johnson, and Alfred Anders..."


And here we have the latest school shootings in our dataset, the latest being Vancouver (at the time of writing).

In [5]:
df.tail()

,Location,Date,Deaths,Injuries,Description
264,"Santa Rosa, California","October 22, 2019",0,1,"A 16-year-old student was shot twice, at least..."
265,"Santa Clarita, California","November 14, 2019",3,3,Saugus High School shooting: Five people were ...
266,"Pleasantville, New Jersey","November 15, 2019",1,2,Five men opened fire during a playoff game bet...
267,"Union City, California","November 23, 2019",2,0,"Two boys, aged 11 and 14-years-old were shot i..."
268,"Vancouver, Washington","November 27, 2019",2,1,Tiffany Hill who was fatally shot by her estra...


### Parsing unsuccessful school shootings
We may get relevant information from unsuccessful school shootings as well. Though we don't expect there to be much news coverage regarding unsuccessful shootings, we can still get more data about factors relating to the event since they would still be relevant in determining potential causes in the shootings. 

In [6]:
def parseWiki(url):
    school_shooting_url = rq.get(url).text
    soup = BeautifulSoup(school_shooting_url)
    soup.prettify()
    
    # It's called a wikitable sortable here, for whatever reason.
    tables = soup.find_all("table",{"class":"wikitable sortable"})
    header = []
    body = []
    # wikipedia separates by year chunks so we iterate through
    for table in tables:
        
        for row in table.find_all("tr"):
            temp = []
            if not header:
                for h in row.find_all("th"):
                    header.append(h.get_text().rstrip())
            for col in row.find_all("td"):
                cur = col.get_text()
                # want to get rid of wiki references
                cur = re.sub(r'\[.*\]', '', cur)
                cur = cur.replace("\n", " ")
                noline = cur.rstrip()
                temp.append(noline)
            body.append(temp)
        
    # the first is the header, so we take it out
    return header, body[1:]


header2, body2 = parseWiki("https://en.wikipedia.org/wiki/List_of_unsuccessful_attacks_related_to_schools")
unsuccessful_shootings = makeDF(header2, body2)

These are some of the earliest unsuccessful attacks but date to before our 2000 for the successful ones.

In [7]:
unsuccessful_shootings.head()

,Date,Location,Description
0,"October 12, 1992","Lincoln, Nebraska, United States","A 43-year-old graduate student, Arthur McElroy..."
1,"November 16, 1998","Burlington, Wisconsin, United States",Five students were arrested the morning they w...
2,May 1999,"Port Huron, Michigan, United States","A 12-year-old, 13-year-old and two 14-year-old..."
3,"January 30, 2001","Cupertino, California, United States",De Anza College student Al DeGuzman planned a ...
4,"February 14, 2001","Elmira, New York, United States","Jeremy Getman, 18, planned a school attack at ..."


These are the most recent unsuccessful attacks.

In [8]:
unsuccessful_shootings.tail()

,Date,Location,Description
24,"May 1, 2014","Waseca, Minnesota, United States",17-year-old John David LaDue
25,"November 3, 2014","Newcastle upon Tyne, United Kingdom",18-year-old Liam Lyburd was arrested at his ho...
26,"December 1, 2014","Plain City, Utah, United States",A 16-year-old at Fremont High School was arres...
27,"February 14, 2018","Everett, Washington, United States","An 18-year-old, Joshua O'Connor, was arrested ..."
28,"April 17, 2019","Jefferson County, Colorado, United States","An 18-year-old, Sol Pais, flew to Colorado fro..."


Now for convenience, we want to convert all the dates in the tables we have to a datetime index. We also have to correct typos Wikipedia has made... because Wikipedia doesn't know how to spell 'February'.

In [9]:
def convert_to_datetime(df):
    for i, row in df.iterrows():
        # Sometimes Wikipedia spells February wrong
        if 'Febuary' in row['Date']:
            row['Date'] = 'February '+" ".join(row['Date'].split()[1:])
        try:
            row['Date'] = datetime.strptime(row['Date'], '%B %d, %Y')
        except:
            row['Date'] = datetime.strptime(row['Date'], '%B %Y')

In [10]:
convert_to_datetime(df)
df.head()

,Location,Date,Deaths,Injuries,Description
0,"Flint, Michigan",2000-02-29 00:00:00,1,0,Shooting of Kayla Rolland: At Buell Elementary...
1,"Lake Worth, Florida",2000-05-26 00:00:00,1,0,"13-year-old honor student, Nathaniel Brazill, ..."
2,"Seattle, Washington",2000-06-28 00:00:00,2,0,58-year-old Director of the Division of Pathol...
3,"Fayetteville, Arkansas",2000-08-28 00:00:00,2,0,"36-year-old James Easton Kelly, a PhD candidat..."
4,"New Orleans, Louisiana",2000-09-26 00:00:00,0,2,"13 year-olds Darrel Johnson, and Alfred Anders..."


In [11]:
convert_to_datetime(unsuccessful_shootings)
unsuccessful_shootings.head()

,Date,Location,Description
0,1992-10-12 00:00:00,"Lincoln, Nebraska, United States","A 43-year-old graduate student, Arthur McElroy..."
1,1998-11-16 00:00:00,"Burlington, Wisconsin, United States",Five students were arrested the morning they w...
2,1999-05-01 00:00:00,"Port Huron, Michigan, United States","A 12-year-old, 13-year-old and two 14-year-old..."
3,2001-01-30 00:00:00,"Cupertino, California, United States",De Anza College student Al DeGuzman planned a ...
4,2001-02-14 00:00:00,"Elmira, New York, United States","Jeremy Getman, 18, planned a school attack at ..."


### Using APIs to get news sources and attempt to parse data
Now that we have the wikipedia data parsed, we want to set up the data scraping from various sources. We have to determine certain attributes of the city that we want to consider when trying to figure out what factors cause the city to have a higher likelyhood of having a school shooting. 

Because it would be extremely hard to query a search on a shooting, we will first take the cities that the school shootings were located in and find the associated wikipedia article. This indicator will be whether that city mentions the shootings as a relevant factor when identifying the city.

We will first look at wikipedia and see if the city that is associated with the shooting ever mentions it. Using the keywords "shooting" and "shooter", we will search through the html of the article. Using "shooting" could also end up catching a movie shooting too, but for the few notable cities that we checked, there weren't any particular references to a movie shooting and only for a school shooting.

In [ ]:
# df: the dataframe that has date, location, injuries, deaths, description of shooting
def trace(df):
    #convert_to_datetime(df)
    city_mentions = []
    total = 0
    ignored = False
    for i, row in df.iterrows():
        '''try:
            date = datetime.strptime(row['Date'], '%B %d, %Y')
        except:
            ignored = True'''
        date = row['Date']
        loc = (row["Location"]).rstrip().replace(" ", "_")
        url = "https://en.wikipedia.org/wiki/" + loc
        city_url = rq.get(url).text
        soup = BeautifulSoup(city_url)
        soup.prettify()
        # seeing if shooting is mentioned
        shooting_results = soup.body.find(string = re.compile(".*shooting.*"))
        # seeing if shooter is mentioned
        shooter_results = soup.body.find(string = re.compile(".*shooter.*"))
        if shooting_results or shooter_results:
            city_mentions.append(True)
        elif ignored:
            city_mentions.append("N/A")
            ignored = False
        else:
            city_mentions.append(False)
    df["City Mentioned"] = city_mentions
    return df
df3 = trace(df)

In [ ]:
df3.head()

In [ ]:
df3.tail()

### Getting other attributes of the school shootings
We can get the population of each of the cities by parsing the html from each of the cities. We have the link to the cities since wikipedia has the format of CITY,_STATE. We can go through each of the rows in the informatino box to find the population. We can also leverage this code to find other important factors in the information box. 

In [ ]:
# Currently this function takes in one url.
def parse_city(url):
    city_pop = ""
    city_url = rq.get(url).text
    soup = BeautifulSoup(city_url)
    #print(soup.prettify())
    texts = soup.find_all("tr")
    nxt = False
    count = 0
    for link in texts:
        if nxt:
            #if 
            pop = link.td.get_text()
            pop = pop.replace(u'\xa0', u' ')
            pop = pop.split(' ')[0]
            if pop[-1] == "]":
                pop = pop[0:len(pop)-3]
            return pop
        if link.get_text()[0:10] == "Population":
            nxt = True

Here is a list of examples of the code chunk above, and you can verify the Populations on the website if you'd like.

In [ ]:
print(parse_city("https://en.wikipedia.org/wiki/Flint,_Michigan"))
print(parse_city("https://en.wikipedia.org/wiki/Lake_Worth_Beach,_Florida"))
print(parse_city("https://en.wikipedia.org/wiki/New_Orleans"))
print(parse_city("https://en.wikipedia.org/wiki/Riverview,_Florida"))
print(parse_city("https://en.wikipedia.org/wiki/Blountsville,_Alabama"))
print(parse_city("https://en.wikipedia.org/wiki/Fayetteville,_Arkansas"))
print(parse_city("https://en.wikipedia.org/wiki/Arapahoe_County,_Colorado"))


In [ ]:
def add_population(df):
    city_populations = []
    for i, row in df.iterrows():
        #Blounstville, Alabama is misspelled in their table.
        if row['Location'] == "Blounstville, Alabama":
            loc = ("Blountsville, Alabama").rstrip().replace(" ", "_")
            url = "https://en.wikipedia.org/wiki/" + loc
            pop = parse_city(url)
            city_populations.append(pop)
        elif row['Location'] is not None:
            loc = (row["Location"]).rstrip().replace(" ", "_")
            url = "https://en.wikipedia.org/wiki/" + loc
            pop = parse_city(url)
            city_populations.append(pop)
        else:
            city_populations.append('None')
    df['Population'] = city_populations
    return df

masterdf = add_population(df3)

In [ ]:
masterdf

### Visualizing our dataframe
We can now plot our relevant data to find potential correlation between several different factors. We want to see if the graphs show us any signifance in terms of the relationship between the x and y axis that we choose for each plot.

In [ ]:
def plot_data(data):
    count = [0]*20
    years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, \
             2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
    casualties = [0]*20
    for i in range(len(data)):
        row = data.iloc[i]
        count[int(row["Date"].year) % 2000] += 1
        casualties[int(row["Date"].year) % 2000] += (int(row["Deaths"]) + int(row["Injuries"]))
    f = {"Year": years, "Number of Occurences": count, "Number of Deaths and Injuries": casualties}
    d = pd.DataFrame(f)
    fig = px.scatter(d, x="Year", y="Number of Occurences", size="Number of Deaths and Injuries", title="School Shootings")
    return fig
fig = plot_data(masterdf)
fig.update_xaxes(zeroline=False)
fig.update_yaxes(zeroline=False)
fig.show()

In [ ]:
def plot_data2(data, bad=True):
    count = [0]*28
    years = [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, \
             2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, \
             2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
    #print(data)
    for i in range(len(data)):
        row = data.iloc[i]
        if int(row["Date"].year) < 2000:
            count[int(row["Date"].year) - 1992] += 1
        else:
            count[(int(row["Date"].year) % 2000) + 8] += 1
    f = {"Year": years, "Number of Occurences": count}
    d = pd.DataFrame(f)
    fig = px.scatter(d, x="Year", y="Number of Occurences", title='Unnsuccessful attacks related to schools')
    return fig
fig = plot_data2(unsuccessful_shootings)
fig.update_xaxes(zeroline=False)
fig.update_yaxes(zeroline=False)
fig.show()

Here we plot the data of school shootings, where the color represents whether or not the shooting was mentioned on the city's Wiki page. In other words, whether or not the shooting(s) help to define the city. We plot the indices of the shootings on the x-axis.

In [ ]:
def plot_results(data):
    casualties = [0]*269
    pop = [0]*269
    mention = [0]*269
    for i in range(len(data)):
        row = data.iloc[i]
        casualties[i] += (int(row["Deaths"]) + int(row["Injuries"]))
        if row["City Mentioned"]:
            mention[i] = "True"
        else:
            mention[i] = "False"
    f = {"Number of Deaths and Injuries": casualties, \
         "Shooting Mentioned on Page": mention, "Indexed Shootings" : range(0, 269),\
        "Pop":data["Population"].tolist()}
    d = pd.DataFrame(f)
    fig = px.scatter(d, x="Indexed Shootings", y="Pop", size="Number of Deaths and Injuries",\
                     title="School Shootings", color="Shooting Mentioned on Page")
    return fig
fig = plot_results(masterdf)
fig.show()

We can see that this graph is extremely cluttered and doesn't tell us much, since Population is essentially right-skew. Thus we correct the upward straggle with a log transform.

In [ ]:
def plot_results(data):
    casualties = [0]*269
    pop = [0]*269
    mention = [0]*269
    for i in range(len(data)):
        row = data.iloc[i]
        casualties[i] += (int(row["Deaths"]) + int(row["Injuries"]))
        nums = row["Population"].split(",")
        num = ""
        for j in nums:
            num += j
        num = int(num)
        pop[i] = np.log(num)
        if row["City Mentioned"]:
            mention[i] = "True"
        else:
            mention[i] = "False"
    f = {"Number of Deaths and Injuries": casualties, \
         "Shooting Mentioned on Page": mention, "Indexed Shootings" : range(0, 269),\
        "logPop":pop}
    d = pd.DataFrame(f)
    #print(d)
    fig = px.scatter(d, x="Indexed Shootings", y="logPop", size="Number of Deaths and Injuries",\
                     title="School Shootings", color="Shooting Mentioned on Page")
    return fig
fig = plot_results(masterdf)
fig.show()

# Scraping New York Times Articles

Now that we have significant numerical data on these events, let's gauge the impact these events have had on society. We're going to measure societal significance by media coverage of the event, primarily focusing on the number of articles published regarding the event. A larger number of articles published reflects a larger societal impact, while a small number of articles reflects a smaller societal impact. In order to consider the initial and immediate impact, we will only be looking at articles published within a month of the event date.

In [ ]:
def format_date(date):
    #format datetime date to be a YYYYMMDD integer
    if date.month < 10 and date.day<10:
        date = '%d0%d%0d'%(date.year,date.month,date.day)
        return int(date)
    elif date.month < 10:
        date = '%d0%d%d'%(date.year,date.month,date.day)
        return int(date)
    elif date.month < 10:
        date = '%d%d0%d'%(date.year,date.month,date.day)
        return int(date)
    else:
        date = '%d%d%d'%(date.year,date.month,date.day)
        return int(date)

def count_articles(location,event_date):
    #start_date = datetime.strptime(event_date,'%B %d, %Y')
    end_date = event_date + timedelta(days=30)
    
    start_date = format_date(event_date)
    end_date = format_date(end_date)
    
    api_key = open('nyt_api_key.txt', 'r').readlines()[0]
    query = '%s School Shooting'%location
    
    url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q= %s&api-key=%s&begin_date=%d&end_date=%d'%(query,api_key,start_date,end_date)
    
    request = rq.get(url)
    json_data = request.json()
    
    # failsafe in case too many requests made
    if 'response' not in json_data.keys():
        print('sleeping...')
        time.sleep(15)
        return count_articles(location,event_date)
    num_articles = json_data['response']['meta']['hits']
    return num_articles
    

In [ ]:
def add_article_data(df):
    article_data = []
    for i, row in df.iterrows():
        location = row['Location']
        event_date = row['Date']
        print(location,event_date)
        num_articles = count_articles(location,event_date)
        article_data.append(num_articles)
        time.sleep(2)
    df['Number of Articles Published'] = article_data

add_article_data(school_shootings)
    